In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerStdio

import os

server = MCPServerStdio(  
    'npx',
        args=[
          "-y",
          "@supabase/mcp-server-supabase@latest",     
          "--access-token",
          os.environ.get("SUPABASE_ACCESS_TOKEN")
        ]
)
agent = Agent(None, mcp_servers=[server])


In [18]:
from pydantic_ai import Agent as PydanticAgent
pydantic_agent = PydanticAgent(None, mcp_servers=[server])

In [31]:
server

MCPServerStdio(command='npx', args=['-y', '@supabase/mcp-server-supabase@latest', '--access-token', 'sbp_f10267332359e54d5cb6435ba796021455e186a6'], env=None, cwd=None)

In [30]:
pydantic_agent._mcp_servers

[MCPServerStdio(command='npx', args=['-y', '@supabase/mcp-server-supabase@latest', '--access-token', 'sbp_f10267332359e54d5cb6435ba796021455e186a6'], env=None, cwd=None)]

In [29]:
pydantic_agent.run_mcp_servers??

Signature: pydantic_agent.run_mcp_servers() -> 'AsyncIterator[None]'
Source:   
    @asynccontextmanager
    async def run_mcp_servers(self) -> AsyncIterator[None]:
        """Run [`MCPServerStdio`s][pydantic_ai.mcp.MCPServerStdio] so they can be used by the agent.

        Returns: a context manager to start and shutdown the servers.
        """
        exit_stack = AsyncExitStack()
        try:
            for mcp_server in self._mcp_servers:
                await exit_stack.enter_async_context(mcp_server)
            yield
        finally:
            await exit_stack.aclose()
File:      ~/Documents/assemblyai/mcp-livekit-agent/aai/lib/python3.11/site-packages/pydantic_ai/agent.py
Type:      method

In [28]:
async with pydantic_agent.run_mcp_servers():
    print(server.is_running)

print(server.is_running)


True
False


In [13]:
hello = server.client_streams()
hello.func(server)

<async_generator object MCPServerStdio.client_streams at 0x1052d7240>

In [6]:
await server._client.initialize()

AttributeError: 'MCPServerStdio' object has no attribute '_client'

In [78]:

import importlib.metadata

def get_package_info(obj):
    module = obj.__module__.split('.')[0]
    try:
        metadata = importlib.metadata.metadata(module)
        return dict(metadata)
    except Exception:
        return None
get_package_info(server)

{'Metadata-Version': '2.4',
 'Name': 'pydantic-ai',
 'Version': '0.1.2',
 'Summary': 'Agent Framework / shim to use Pydantic with LLMs',
 'Project-URL': 'Homepage, https://ai.pydantic.dev',
 'Author-email': 'Samuel Colvin <samuel@pydantic.dev>, Marcelo Trylesinski <marcelotryle@gmail.com>, David Montague <david@pydantic.dev>, Alex Hall <alex@pydantic.dev>',
 'License-Expression': 'MIT',
 'License-File': 'LICENSE',
 'Classifier': 'Development Status :: 4 - Beta',
 'Requires-Python': '>=3.9',
 'Requires-Dist': 'pydantic-ai-slim[anthropic,bedrock,cli,cohere,evals,groq,mcp,mistral,openai,vertexai]==0.1.2',
 'Provides-Extra': 'examples',
 'Description-Content-Type': 'text/markdown',
 'Description': '<div align="center">\n  <a href="https://ai.pydantic.dev/">\n    <picture>\n      <source media="(prefers-color-scheme: dark)" srcset="https://ai.pydantic.dev/img/pydantic-ai-dark.svg">\n      <img src="https://ai.pydantic.dev/img/pydantic-ai-light.svg" alt="PydanticAI">\n    </picture>\n  </a>\

In [64]:
await server.list_tools()

[ToolDefinition(name='list_projects', description='Lists all Supabase projects for the user.', parameters_json_schema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outer_typed_dict_key=None, strict=None),
 ToolDefinition(name='get_project', description='Gets details for a Supabase project.', parameters_json_schema={'type': 'object', 'properties': {'id': {'type': 'string', 'description': 'The project ID'}}, 'required': ['id'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outer_typed_dict_key=None, strict=None),
 ToolDefinition(name='get_cost', description='Gets the cost of creating a new project or branch. Never assume organization as costs can be different for each.', parameters_json_schema={'type': 'object', 'properties': {'type': {'type': 'string', 'enum': ['project', 'branch']}, 'organization_id': {'type': 'string', 'description': 'The organization ID. Always ask the

In [60]:
with await server.list_tools()
    print("Server: ", server.is_running)
    print("Server: ", server.to_dict())
    print("Server connected: ", server.is_connected)
    print("Server connected: ", server.is_running)
    print("Server connected: ", server.is_connected)
    print("Server connected: ", server.is_running)

SyntaxError: expected ':' (55097386.py, line 1)

In [10]:
from livekit.agents import (
    Agent,
    AgentSession,
    JobContext,
    RunContext,
    WorkerOptions,
    cli,
    function_tool,
)
@function_tool
async def lookup_weather(
    context: RunContext,
    location: str,
):
    """Used to look up weather information."""

    return {"weather": "sunny", "temperature": 70}


In [27]:
from __future__ import annotations

import json
import os
from typing import List, Callable

from dotenv import load_dotenv
from livekit.agents import (
    Agent,
    AgentSession,
    JobContext,
    RunContext,
    WorkerOptions,
    cli,
    function_tool,
    FunctionTool
)
from livekit.plugins import assemblyai, openai, silero
from pydantic_ai import Agent as PydanticAgent
from pydantic_ai.mcp import MCPServerStdio

load_dotenv()

from typing import List, Callable, Any  # add Any
def _py_type(s: dict) -> str:
    t = s.get("type")
    if t == "string":
        return "str"
    if t in ("number", "integer"):
        return "float" if t == "number" else "int"
    if t == "boolean":
        return "bool"
    if t == "array":
        return f"List[{_py_type(s.get('items', {}))}]"
    if t == "object":
        return "dict"
    return "Any"


# --------------------------------------------------------------------------- #
# Helper: convert every MCP ToolDefinition into a LiveKit function_tool
# --------------------------------------------------------------------------- #
async def build_livekit_tools(server) -> List[Callable]:
    tools: List[Callable] = []

    for td in await server.list_tools():
        props = td.parameters_json_schema.get("properties", {})
        required = set(td.parameters_json_schema.get("required", []))

        # build google‑style docstring
        doc = [td.description or ""]
        if props:
            doc.append("\nArgs:")
            for name, schema in props.items():
                typ = _py_type(schema)
                req = "required" if name in required else "optional"
                desc = schema.get("description", "")
                doc.append(f"    {name} ({typ}, {req}): {desc}")
        full_doc = "\n".join(doc).rstrip()

        async def _proxy(context: RunContext, _td=td, **kwargs):
            result = await server.call_tool(_td.name, arguments=kwargs or None)
            text = result.content[0].text
            try:
                return json.loads(text)
            except Exception:
                return text

        _proxy.__name__ = td.name
        _proxy.__doc__ = full_doc
        tools.append(function_tool(_proxy))

    return tools

In [46]:
import types

def template(y: int):
    return x + y  # x is not defined yet

# Create new function with globals that define x
new_func = types.FunctionType(template.__code__, {'x': 10})
print(new_func(5))  # Output: 15

new_func.__annotations__

15


{}

In [50]:
first_tool = tools[0]

first_tool()

TypeError: build_livekit_tools.<locals>._proxy() missing 1 required positional argument: 'context'

In [7]:
async def main():
    async with agent.run_mcp_servers():
        livekit_tools = await build_livekit_tools(server)
        for tool in livekit_tools:
            print(tool)

        return livekit_tools, None
        tools = await server.list_tools()

        result = await server.call_tool("list_projects", arguments = None)
        print(result)
        return tools, result

tools, result= await main()

<function build_livekit_tools.<locals>._proxy at 0x11aee1da0>
<function build_livekit_tools.<locals>._proxy at 0x11aee2660>
<function build_livekit_tools.<locals>._proxy at 0x11aee2700>
<function build_livekit_tools.<locals>._proxy at 0x11aee27a0>
<function build_livekit_tools.<locals>._proxy at 0x11aee2840>
<function build_livekit_tools.<locals>._proxy at 0x11aee2a20>
<function build_livekit_tools.<locals>._proxy at 0x11aee2ac0>
<function build_livekit_tools.<locals>._proxy at 0x11aee2d40>
<function build_livekit_tools.<locals>._proxy at 0x11aee2b60>
<function build_livekit_tools.<locals>._proxy at 0x11aee2e80>
<function build_livekit_tools.<locals>._proxy at 0x11aee2de0>
<function build_livekit_tools.<locals>._proxy at 0x11aee2f20>
<function build_livekit_tools.<locals>._proxy at 0x11aee2fc0>
<function build_livekit_tools.<locals>._proxy at 0x11aee3060>
<function build_livekit_tools.<locals>._proxy at 0x11aee3100>
<function build_livekit_tools.<locals>._proxy at 0x11aee31a0>
<functio

AttributeError: type object 'FunctionTool' has no attribute 'from_function_tool'

In [16]:
for tool in tools:
    print(tool.__name__)
    print(tool.__doc__)
    print(tool.__annotations__)
    print("="*100)

list_projects
Lists all Supabase projects for the user.
{'context': 'RunContext'}
get_project
Gets details for a Supabase project.

Args:
    id (str, required): The project ID
{'context': 'RunContext'}
get_cost
Gets the cost of creating a new project or branch. Never assume organization as costs can be different for each.

Args:
    type (str, required): 
    organization_id (str, required): The organization ID. Always ask the user.
{'context': 'RunContext'}
confirm_cost
Ask the user to confirm their understanding of the cost of creating a new project or branch. Call `get_cost` first. Returns a unique ID for this confirmation which should be passed to `create_project` or `create_branch`.

Args:
    type (str, required): 
    recurrence (str, required): 
    amount (float, required):
{'context': 'RunContext'}
create_project
Creates a new Supabase project. Always ask the user which organization to create the project in. The project can take a few minutes to initialize - use `get_project

In [33]:
def livekit_tool_from_function_tool(pydantic_tool, server):

    async def tool_fn(**kwargs):
        return await server.call_tool(pydantic_tool.name, kwargs)
    
    tool_fn.__name__ = pydantic_tool.name
    tool_fn.__doc__ = pydantic_tool.description or ""
    tool_fn.__annotations__ = pydantic_tool.parameters_json_schema.__annotations__

    return function_tool(f= tool_fn, name=pydantic_tool.name, description=pydantic_tool.description)
    

In [77]:
tools[3].parameters_json_schema

def schema_to_docstring(schema: dict, description: str, indent: int = 4) -> str:
    def render_property(name: str, prop: dict, is_required: bool, current_indent: int) -> str:
        space = " " * current_indent
        type_ = prop.get("type", "any")
        req = " - required" if is_required else ""
        field_desc = prop.get("description", "").strip()
        
        # If the property is an object with nested properties, render them recursively.
        if type_ == "object" and "properties" in prop:
            nested_required = set(prop.get("required", []))
            sub_fields = []
            for sub_name, sub_prop in prop["properties"].items():
                sub_line = render_property(sub_name,
                                           sub_prop,
                                           sub_name in nested_required,
                                           current_indent + indent)
                sub_fields.append(sub_line)
            nested_doc = "\n".join(sub_fields)
            return f"{space}{name} (object{req}): {field_desc}\n{nested_doc}"
        # If the property is an array, check if items is an object to possibly render nested fields.
        elif type_ == "array":
            items = prop.get("items", {})
            # If the items are objects with properties, render a summary of the subfields.
            if items.get("type") == "object" and "properties" in items:
                nested_required = set(items.get("required", []))
                sub_fields = []
                for sub_name, sub_prop in items["properties"].items():
                    sub_line = render_property(sub_name,
                                               sub_prop,
                                               sub_name in nested_required,
                                               current_indent + indent)
                    sub_fields.append(sub_line)
                nested_doc = "\n".join(sub_fields)
                type_str = f"array of objects"
                return f"{space}{name} ({type_str}{req}): {field_desc}\n{nested_doc}"
            else:
                # For simple arrays, describe the type of items if possible.
                item_type = items.get("type", "any")
                type_str = f"array of {item_type}"
                return f"{space}{name} ({type_str}{req}): {field_desc}"
        else:
            # Base case: primitive types.
            return f"{space}{name} ({type_}{req}): {field_desc}"
    
    # Process the top-level properties.
    properties = schema.get("properties", {})
    required_fields = set(schema.get("required", []))
    args_lines = []
    for prop_name, prop in properties.items():
        line = render_property(prop_name, prop, prop_name in required_fields, indent)
        args_lines.append(line)

    args_doc = "\n".join(args_lines)
    return f'"""{description}\n\nArgs:\n{args_doc}\n"""'
i = 9
print(tools[i].description)
print(tools[i].parameters_json_schema)
t = schema_to_docstring(tools[i].parameters_json_schema, tools[i].description)
print(t)

Lists all tables in a schema.
{'type': 'object', 'properties': {'project_id': {'type': 'string'}, 'schemas': {'type': 'array', 'items': {'type': 'string'}, 'description': 'Optional list of schemas to include. Defaults to all schemas.'}}, 'required': ['project_id'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}
"""Lists all tables in a schema.

Args:
    project_id (string - required): 
    schemas (array of string): Optional list of schemas to include. Defaults to all schemas.
"""


In [61]:
tools[10].parameters_json_schema

{'type': 'object',
 'properties': {'project_id': {'type': 'string'}},
 'required': ['project_id'],
 'additionalProperties': False,
 '$schema': 'http://json-schema.org/draft-07/schema#'}

In [34]:
livekit_tool_from_function_tool(tools[1], server)

AttributeError: 'dict' object has no attribute '__annotations__'

In [28]:
from livekit.agents import (
    Agent,
    function_tool,
    FunctionTool,
)
function_tool()

In [20]:
tool_definition = tools[1]

In [25]:
tool_definition.name

'get_project'

In [26]:
tool_definition.description

'Gets details for a Supabase project.'

In [24]:
tool_definition.parameters_json_schema

{'type': 'object',
 'properties': {'id': {'type': 'string', 'description': 'The project ID'}},
 'required': ['id'],
 'additionalProperties': False,
 '$schema': 'http://json-schema.org/draft-07/schema#'}

In [ ]:
function_tool(name=tool_definition.name,
             description=tool_definition.description, parameters_json_schema=tool_definition.parameters_json_schema)


TypeError: function_tool() got an unexpected keyword argument 'parameters_json_schema'

In [21]:
tool_definition.parameters_json_schema

{'type': 'object',
 'properties': {'id': {'type': 'string', 'description': 'The project ID'}},
 'required': ['id'],
 'additionalProperties': False,
 '$schema': 'http://json-schema.org/draft-07/schema#'}

In [6]:
tools

[ToolDefinition(name='list_projects', description='Lists all Supabase projects for the user.', parameters_json_schema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outer_typed_dict_key=None, strict=None),
 ToolDefinition(name='get_project', description='Gets details for a Supabase project.', parameters_json_schema={'type': 'object', 'properties': {'id': {'type': 'string', 'description': 'The project ID'}}, 'required': ['id'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outer_typed_dict_key=None, strict=None),
 ToolDefinition(name='get_cost', description='Gets the cost of creating a new project or branch. Never assume organization as costs can be different for each.', parameters_json_schema={'type': 'object', 'properties': {'type': {'type': 'string', 'enum': ['project', 'branch']}, 'organization_id': {'type': 'string', 'description': 'The organization ID. Always ask the

In [34]:
print(tools[0])

ToolDefinition(name='list_projects', description='Lists all Supabase projects for the user.', parameters_json_schema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outer_typed_dict_key=None, strict=None)


In [26]:
tools[10]

ToolDefinition(name='list_extensions', description='Lists all extensions in the database.', parameters_json_schema={'type': 'object', 'properties': {'project_id': {'type': 'string'}}, 'required': ['project_id'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outer_typed_dict_key=None, strict=None)

In [3]:
async with MCPServerStdio(
    params={
        "command": "npx",
        "args": [
             "-y",
              "@supabase/mcp-server-supabase@latest",
              "--access-token",
              os.environ.get("SUPABASE_ACCESS_TOKEN")
                ],
    }
) as server:
    tools = await server.list_tools()